In [10]:
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition
Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)
Input may optionally be inverted, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.
Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs
Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs
Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs
Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

from __future__ import print_function
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Activation
import numpy as np
from six.moves import range


class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).

model.add(Dense(len(chars), activation='relu',input_shape=(MAXLEN, len(chars))))

model.add(RNN(HIDDEN_SIZE))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary() #[][]

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 50):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if INVERT else q)
        print('T', correct)
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=" ")
        else:
            print(colors.fail + '☒' + colors.close, end=" ")
        print(guess)
        print('---')

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 7, 12)             156       
_________________________________________________________________
lstm_9 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_5 (Activation)    (None,

45000/45000 [==============================] - 13s - loss: 0.2468 - acc: 0.9483 - val_loss: 0.2121 - val_acc: 0.9594
Q 456+96 
T 552 
☑ 552 
---
Q 982+49 
T 1031
☑ 1031
---
Q 451+953
T 1404
☒ 1403
---
Q 827+45 
T 872 
☑ 872 
---
Q 719+61 
T 780 
☑ 780 
---
Q 75+228 
T 303 
☒ 203 
---
Q 635+85 
T 720 
☑ 720 
---
Q 1+697  
T 698 
☒ 798 
---
Q 149+301
T 450 
☒ 440 
---
Q 629+40 
T 669 
☑ 669 
---

--------------------------------------------------
Iteration 13
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 13s - loss: 0.1817 - acc: 0.9673 - val_loss: 0.1759 - val_acc: 0.9617
Q 502+975
T 1477
☑ 1477
---
Q 692+54 
T 746 
☑ 746 
---
Q 920+448
T 1368
☑ 1368
---
Q 45+482 
T 527 
☑ 527 
---
Q 9+411  
T 420 
☑ 420 
---
Q 84+701 
T 785 
☑ 785 
---
Q 103+25 
T 128 
☑ 128 
---
Q 6+987  
T 993 
☑ 993 
---
Q 216+44 
T 260 
☑ 260 
---
Q 616+343
T 959 
☑ 959 
---

--------------------------------------------------
Iteration 14
Train on 45000 sa

45000/45000 [==============================] - 12s - loss: 0.0157 - acc: 0.9985 - val_loss: 0.0210 - val_acc: 0.9960
Q 7+63   
T 70  
☒ 60  
---
Q 3+166  
T 169 
☑ 169 
---
Q 943+27 
T 970 
☑ 970 
---
Q 213+496
T 709 
☑ 709 
---
Q 312+15 
T 327 
☑ 327 
---
Q 2+509  
T 511 
☑ 511 
---
Q 977+340
T 1317
☑ 1317
---
Q 8+988  
T 996 
☑ 996 
---
Q 93+162 
T 255 
☑ 255 
---
Q 903+768
T 1671
☑ 1671
---

--------------------------------------------------
Iteration 27
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s - loss: 0.0192 - acc: 0.9966 - val_loss: 0.0723 - val_acc: 0.9758
Q 172+862
T 1034
☒ 1035
---
Q 59+675 
T 734 
☑ 734 
---
Q 32+808 
T 840 
☑ 840 
---
Q 200+760
T 960 
☑ 960 
---
Q 804+119
T 923 
☑ 923 
---
Q 229+461
T 690 
☑ 690 
---
Q 8+343  
T 351 
☑ 351 
---
Q 649+63 
T 712 
☑ 712 
---
Q 844+448
T 1292
☑ 1292
---
Q 240+156
T 396 
☑ 396 
---

--------------------------------------------------
Iteration 28
Train on 45000 sa

45000/45000 [==============================] - 12s - loss: 0.0048 - acc: 0.9997 - val_loss: 0.0070 - val_acc: 0.9983
Q 86+832 
T 918 
☑ 918 
---
Q 79+74  
T 153 
☑ 153 
---
Q 75+675 
T 750 
☑ 750 
---
Q 772+366
T 1138
☑ 1138
---
Q 58+559 
T 617 
☑ 617 
---
Q 7+600  
T 607 
☑ 607 
---
Q 19+78  
T 97  
☑ 97  
---
Q 30+539 
T 569 
☑ 569 
---
Q 232+60 
T 292 
☑ 292 
---
Q 838+46 
T 884 
☑ 884 
---

--------------------------------------------------
Iteration 41
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s - loss: 0.0042 - acc: 0.9997 - val_loss: 0.0067 - val_acc: 0.9989
Q 592+37 
T 629 
☑ 629 
---
Q 18+282 
T 300 
☑ 300 
---
Q 90+716 
T 806 
☑ 806 
---
Q 593+387
T 980 
☑ 980 
---
Q 45+983 
T 1028
☑ 1028
---
Q 121+828
T 949 
☑ 949 
---
Q 840+453
T 1293
☑ 1293
---
Q 96+190 
T 286 
☑ 286 
---
Q 56+179 
T 235 
☑ 235 
---
Q 44+22  
T 66  
☑ 66  
---

--------------------------------------------------
Iteration 42
Train on 45000 sa

In [2]:
query = "123 + 456   "
query = query[::-1]

In [3]:
query

'   654 + 321'

In [4]:
model?